In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import os, glob, random, cv2, glob, pydicom


In [ ]:

directory = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/train'
vox = ["FLAIR","T1w", "T1wCE", "T2w"]
 

## Determining Planes

In [ ]:
# Get a list of images in a series
def get_series_list(case, series_name):
    series_list = []
    for subdirs, dirs, files in os.walk(directory + '/' + case + "/" + series_name):
        series_list = os.listdir(directory + '/' + case + '/' + series_name)        
    return series_list

In [ ]:
# Find the image in the middle of each series stack, returns a list of four images
def get_middle_images(case):
    
    middle_images = []
    
    # Iterate through each of the four series directories and get the files 
    for ser in vox:
        series_files = get_series_list(case, ser)
        series_df = pd.DataFrame(columns = ['image','instance_number'])

        # Get the DICOM InstanceNumber tag to order the images since we can't rely on the filenames to be in order
        for s in series_files:
            img = pydicom.dcmread(directory + "/" + case + "/" + ser + "/" + s)
            series_df.loc[len(series_df.index)] = [s, img[0x0020,0x0013].value] 
        series_df['instance_number'] = pd.to_numeric(series_df['instance_number'])

        # Sort the image list by InstanceNumber
        series_df = series_df.sort_values(by=['instance_number'])
        
        # Find the image in the middle of the list
        middle_index = int(series_df.shape[0] / 2)
        middle_image = series_df.iloc[middle_index]['image']

        middle_images.append(ser + "/" + middle_image)

    return middle_images

In [ ]:
# Convert the Image Orientation Patient tag cosine values into a text string of the plane.
# This represents the plane the image is 'closest to' .. it does not explain any obliqueness
def get_image_plane(loc):
    row_x = round(loc[0])
    row_y = round(loc[1])
    row_z = round(loc[2])
    col_x = round(loc[3])
    col_y = round(loc[4])
    col_z = round(loc[5])
    if row_x == 1 and row_y == 0 and col_x == 0 and col_y == 0:
        return "Coronal"
    if row_x == 0 and row_y == 1 and col_x == 0 and col_y == 0:
        return "Sagittal"
    if row_x == 1 and row_y == 0 and col_x == 0 and col_y == 1:
        return "Axial"
    return "Unknown"

In [ ]:
# Display images and their plane
#def plane_images(dataset,case):
        
case1=[]
flair,j_flair=[],[]  
t1w,j_t1w=[],[]    
t1wce,j_t1wce=[],[]   
t2w,j_t2w=[],[]    
#note = []

df ={}
#dataset="train"

cases = os.listdir(directory)
for case in cases:
      
    for img in get_middle_images(case):
        
        baca=directory + "/" + case 
        image = pydicom.dcmread(baca+ "/" + img)
        ser = img.split("/")
        image_orientation_patient = image[0x0020,0x0037]        
        plane = get_image_plane(image_orientation_patient)

        if ser[0]=="FLAIR":
            flair.append(plane)   #+'('+str(len(os.listdir(baca+ "/FLAIR")))+')')
            j_flair.append(len(os.listdir(baca+ "/FLAIR")))
        if ser[0]=="T1w":
            t1w.append(plane)   #+'('+str(len(os.listdir(baca+ "/T1w")))+')')
            j_t1w.append(len(os.listdir(baca+ "/T1w")))
        if ser[0]=="T1wCE":
            t1wce.append(plane)   #+'('+str(len(os.listdir(baca+ "/T1wCE")))+')')
            j_t1wce.append(len(os.listdir(baca+ "/T1wCE")))
        if ser[0]=="T2w":
            t2w.append(plane)   #+'('+str(len(os.listdir(baca+ "/T2w")))+')' )            
            j_t2w.append(len(os.listdir(baca+ "/T2w")))
                       
    case1.append(case)   #+'('+str(len(cases))+')')
    #note.append(img)
    
    df = pd.DataFrame.from_dict({        
        "case1" : case1,
        #"0" : jum_case1,
        
        "flair" : flair,
        "j_flair" : j_flair,
        
        "t1w" : t1w,
        "j_t1w" : j_t1w,
        
        "t1wCE" : t1wce,
        "j_t1wce" : j_t1wce,
        
        "t2w" : t2w,
        "j_t2w" : j_t2w
        
        #"Note" : note
    })
    #return df    
print(df)

In [ ]:
# df.to_excel("PreprocessDataMRI-1.xlsx", sheet_name="data", index=False)
# df.to_csv("PreprocessDataMRI-1.csv", index=False)

In [ ]:
df_train_labels_dir = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv'
df_train_labels = pd.read_csv(df_train_labels_dir) 

#df_train_labels = df_train_labels.rename({'BraTS21ID': 'case1'}, axis=1)
df_train_labels['BraTS21ID'] = df_train_labels['BraTS21ID'].apply(lambda x: '{0:0>5}'.format(x))
print(len(df_train_labels))    
df_train_labels.head()

In [ ]:
#df_train=pd.read_csv('PreprocessDataMRI-0.csv')
df_train=df

df_train = df_train.rename({'case1':'BraTS21ID','flair': 'FLAIR','t1w':'T1w','t1wCE':'T1wCE','t2w':'T2w'}, axis=1)
df_train = df_train.rename({'j_flair': 'j_FLAIR','j_t1w':'j_T1w','j_t1wce':'j_T1wCE','j_t2w':'j_T2w'}, axis=1)
df_train['BraTS21ID'] = df_train['BraTS21ID'].apply(lambda x: '{0:0>5}'.format(x))
df_train['path_BraTS21ID']=directory+'/'+df_train['BraTS21ID'] 
df_train2 = pd.concat([df_train, df_train_labels['MGMT_value']], axis=1)
df_train2[0:10]

## Adjust contrast and brightness

In [ ]:
# Make a simple linear VOI LUT from the raw (stored) pixel data
def make_lut(storedPixels, windowWidth, windowLevel, p_i):
    
    # Slope and Intercept set to 1 and 0 for MR. Get these from DICOM tags instead if using 
    # on a modality that requires them (CT, PT etc)
    slope = 1.0
    intercept = 0.0
    minPixel = int(np.amin(storedPixels))
    maxPixel = int(np.amax(storedPixels))

    # Make an empty array for the LUT the size of the pixel 'width' in the raw pixel data
    lut = [0] * (maxPixel + 1)
    
    # Invert pixels and windowLevel for MONOCHROME1. We invert the specified windowLevel so that 
    # increasing the level value makes the images brighter regardless of photometric intrepretation
    invert = False
    if p_i == "MONOCHROME1":
        invert = True
    else:
        windowLevel = (maxPixel - minPixel) - windowLevel
        
    # Loop through the pixels and calculate each LUT value
    for storedValue in range(minPixel, maxPixel):
        modalityLutValue = storedValue * slope + intercept
        voiLutValue = (((modalityLutValue - windowLevel) / windowWidth + 0.5) * 255.0)
        clampedValue = min(max(voiLutValue, 0), 255)
        if invert:
            lut[storedValue] = round(255-clampedValue)
        else:
            lut[storedValue] = round(clampedValue)
        
    return lut

In [ ]:
# Apply the LUT to a pixel array
def apply_lut(pixels_in, lut):
    
    pixels_in = pixels_in.flatten()
    pixels_out = [0] * len(pixels_in)
    
    for i in range(0, len(pixels_in)):
        pixel = pixels_in[i]
        pixels_out[i] = int(lut[pixel])
        
    return pixels_out

In [ ]:
pasient=os.listdir(directory)[0]

#vox = ["FLAIR","T1w", "T1wCE", "T2W"]
def gambar(baris,kolom):  
    t=directory+'/'+pasient+'/'+vox[kolom]
    t_paths = sorted(os.listdir(t), 
            key=lambda x: int(x[:-4].split("-")[-1]),)   
    mid =len(os.listdir(directory+'/'+pasient+'/'+vox[kolom]))//2
    t_paths_=t_paths[mid-((nbaris+2)//2):mid+((nbaris+2)//2)]  
    t_paths_ = t_paths_[baris]
    return t_paths_

window_width = 300    # 1000,600,300,  1500,1000, 700
window_level = 900     #  900,900,900,  2000,2000,2000

nbaris=10

fig,axs = plt.subplots(nbaris,4,figsize=(4*10,nbaris*10))
fig.subplots_adjust(top=.96)
fig.suptitle('pasient = '+pasient, fontsize=60, fontweight='bold')

for kolom in range (4):
    for baris in range(nbaris):     
        gam = pydicom.read_file(directory+'/'+pasient+'/'+vox[kolom]+'/'+gambar(baris,kolom))   #.pixel_array 
        
        lut = make_lut(gam.pixel_array, window_width, window_level, gam.PhotometricInterpretation)
        gam = np.reshape(apply_lut(gam.pixel_array, lut), (gam.pixel_array.shape[0],gam.pixel_array.shape[1]))
        
        plane=df_train2[df_train2['BraTS21ID'] == pasient][vox[kolom]].values[0]
        
        axs[baris, kolom].imshow(gam,cmap="viridis")   #cmap='gray')   #cmap="viridis") 
        axs[baris, kolom].set_title(vox[kolom]+'-'+plane+' ('+gambar(baris,kolom)+')',fontsize=26)

In [ ]:
pasient=os.listdir(directory)[1]


def gambar(baris,kolom):  
    t=directory+'/'+pasient+'/'+vox[kolom]
    t_paths = sorted(os.listdir(t), 
            key=lambda x: int(x[:-4].split("-")[-1]),)   
    mid =len(os.listdir(directory+'/'+pasient+'/'+vox[kolom]))//2
    t_paths_=t_paths[mid-((nbaris+2)//2):mid+((nbaris+2)//2)]  
    t_paths_ = t_paths_[baris]
    return t_paths_

nbaris=7

fig,axs = plt.subplots(nbaris,4,figsize=(4*10,nbaris*10))
fig.subplots_adjust(top=.96)
fig.suptitle('pasient = '+pasient, fontsize=60, fontweight='bold')

for kolom in range (4):
    for baris in range(nbaris):     
        gam = pydicom.read_file(directory+'/'+pasient+'/'+vox[kolom]+'/'+gambar(baris,kolom)).pixel_array           
        axs[baris, kolom].imshow(gam,cmap="viridis") 
        axs[baris, kolom].set_title(vox[kolom]+' ('+gambar(baris,kolom)+')',fontsize=26)
